In [56]:
import numpy as np
import zarr
import matplotlib.pyplot as plt
import napari

In [2]:
%matplotlib qt

In [11]:
# load in a test membrane image. low res
im = np.array(zarr.open(r'/media/brandon/Data1/Brandon/fly_immune/Lightsheet_Z1/2023_02_07_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_earlyL3_24hrs/larvae_1/crop.ome.zarr/4', 'r'))[0,1]

In [101]:
thresh = 2000
mask = im > thresh
#mask = mask[:,200:400,:]

In [55]:
plt.figure()
plt.imshow(mask[50])

In [102]:
zgrid, ygrid, xgrid = np.indices(mask.shape)

In [104]:
z = zgrid[mask]
y = ygrid[mask]
x = xgrid[mask]

In [105]:
random_indices = np.random.randint(len(z), size=4000)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x[random_indices] * 0.3295 * 2**4, y[random_indices] * 0.3295 * 2**4, z[random_indices] * 2, 'ko', s=4)
ax.axes.set_xlim3d(left=0, right=3000)
ax.axes.set_ylim3d(bottom=0, top=3000) 
ax.axes.set_zlim3d(bottom=0, top=3000) 


(0.0, 3000.0)

In [52]:
len(z)

264905

In [32]:
im.shape

(396, 607, 167)

In [106]:
viewer = napari.view_image(im)

In [80]:
points = np.concatenate((np.expand_dims(z, axis=0), np.expand_dims(y, axis=0), np.expand_dims(x, axis=0)),axis=0).T
points[random_indices].shape

(1000, 3)

In [81]:
viewer.add_points(points[random_indices])

/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Points layer 'Points' at 0x7fc79d481850>

In [79]:
points[random_indices].shape

(1000, 3)

In [77]:
viewer.layers[0].data.shape

(396, 607, 167)

In [78]:
points = np.concatenate((np.expand_dims(z, axis=0), np.expand_dims(y, axis=0), np.expand_dims(x, axis=0)),axis=0).T


In [119]:
viewer.layers[0].scale = [2, 0.3295 * 16, 0.3295 * 16]
#viewer.layers[1].scale = [2, 0.3295 * 16, 0.3295 * 16]

IndexError: list index out of range

In [120]:
origin = (0, 0, int(im.shape[2] / 2) * 0.3295 * 16)

In [121]:
origin

(0, 0, 437.576)

In [130]:
xum = x * 0.3295 * 16
yum = y * 0.3295 * 16
zum = z * 2
r = np.sqrt((xum - origin[2]) **2 + (zum - origin[0]) **2)
theta = np.arctan(zum / xum)
v = np.array(theta * 1000, dtype=int)

In [123]:
plt.figure()
plt.plot(yum[random_indices], r[random_indices], 'ko', markersize=4, alpha=0.1)

In [124]:
plt.xlim([0,3000])
plt.ylim([0, 3000])

(0.0, 3000.0)

In [156]:
plt.figure()
plt.subplot(1,2,1)
plt.plot(v[random_indices], yum[random_indices], 'ko', markersize=4, alpha=0.1)
plt.subplot(1,2,2)
plt.plot(xum[random_indices], yum[random_indices], 'ko', markersize=4, alpha=0.1)

In [142]:
Imap = np.zeros((im.shape[1], 2000))

In [143]:
Imap.shape

(607, 2000)

In [144]:
for k in range(im.shape[0]):
    for j in range(im.shape[1]):
        for i in range(im.shape[2]):
            this_theta = np.arctan(zgrid[k,j,i] / xgrid[k,j,i])
            if np.isnan(this_theta):
                continue
            this_v = int(this_theta * 1000)
            Imap[ygrid[k,j,i], this_v] = im[k,j,i]

/tmp/ipykernel_1361560/2554899069.py:4: RuntimeWarning: invalid value encountered in long_scalars
  this_theta = np.arctan(zgrid[k,j,i] / xgrid[k,j,i])
/tmp/ipykernel_1361560/2554899069.py:4: RuntimeWarning: divide by zero encountered in long_scalars
  this_theta = np.arctan(zgrid[k,j,i] / xgrid[k,j,i])


In [150]:
for i in range(len(random_indices)):
    this_x = x[i]
    this_y = y[i]
    this_z = z[i]
    this_theta = np.arctan(this_z / this_x)
    if np.isnan(this_theta):
        print('nan!')
        continue
    this_v = int(this_theta * 1000)
    Imap[this_y, this_v] = im[this_z, this_y, this_x]

In [152]:
plt.figure()
plt.imshow(Imap)